In [1]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import flipside_utils as f
import clickhouse_utils as ch
sys.path.pop()

import numpy as np
import pandas as pd
import os

In [2]:
flipside_configs = [
        {'blockchain': 'blast', 'name': 'Blast', 'layer': 'L2', 'trailing_days': 365}
]
clickhouse_configs = [
        {'blockchain': 'metal', 'name': 'Metal', 'layer': 'L2', 'trailing_days': 365},
        {'blockchain': 'mode', 'name': 'Mode', 'layer': 'L2', 'trailing_days': 365},
        {'blockchain': 'bob', 'name': 'BOB (Build on Bitcoin)', 'layer': 'L2', 'trailing_days': 365},
        {'blockchain': 'fraxtal', 'name': 'Fraxtal', 'layer': 'L2', 'trailing_days': 365},
]

In [3]:
# Run Flipside
flip_dfs = []
with open(os.path.join("inputs/sql/flipside_bychain.sql"), "r") as file:
                        query = file.read()

for chain in flipside_configs:
        #Pass in Params to the query
        query = query.replace("@blockchain@", chain['blockchain'])
        query = query.replace("@name@", chain['name'])
        query = query.replace("@layer@", chain['layer'])
        query = query.replace("@trailing_days@", str(chain['trailing_days']))
        
        df = f.query_to_df(query)

        flip_dfs.append(df)

flip = pd.concat(flip_dfs)
flip['dt'] = pd.to_datetime(flip['dt']).dt.tz_localize(None)
flip = flip[['dt','blockchain','name','layer','num_qualified_txs']]


In [4]:
# Run Dune
dune_df = d.get_dune_data(query_id = 3740822, #https://dune.com/queries/3740822
    name = "dune_evms_qualified_txs",
    path = "outputs"
)
dune_df['dt'] = pd.to_datetime(dune_df['dt']).dt.tz_localize(None)
dune_df = dune_df[['dt','blockchain','name','layer','num_qualified_txs']]

2024-05-17 11:51:10.165 | INFO     | duneapi_utils:get_dune_data:44 - Results available at https://dune.com/queries/3740822
2024-05-17 11:51:11.149 | INFO     | duneapi_utils:get_dune_data:89 - ✨ Results saved as outputs/dune_evms_qualified_txs.csv, with 4541 rows and 6 columns.


In [5]:
# Run Clickhouse
ch_dfs = []
ch_client = ch.connect_to_clickhouse_db() #Default is OPLabs DB

with open(os.path.join("inputs/sql/goldsky_bychain.sql"), "r") as file:
                        query = file.read()

for chain in clickhouse_configs:
        #Pass in Params to the query
        query = query.replace("@blockchain@", chain['blockchain'])
        query = query.replace("@name@", chain['name'])
        query = query.replace("@layer@", chain['layer'])
        query = query.replace("@trailing_days@", str(chain['trailing_days']))
        
        df = ch_client.query_df(query)

        ch_dfs.append(df)

ch = pd.concat(ch_dfs)
ch['dt'] = pd.to_datetime(ch['dt']).dt.tz_localize(None)
ch = ch[['dt','blockchain','name','layer','num_qualified_txs']]

In [6]:
# Step 1: Filter dune_df for chains not in flip
filtered_dune_df = dune_df[~dune_df['blockchain'].isin(flip['blockchain'])]
# Step 2: Union flip and filtered_dune_df
combined_flip_dune = pd.concat([flip, filtered_dune_df])
# Step 3: Filter ch for chains not in combined_flip_dune
filtered_ch = ch[~ch['blockchain'].isin(combined_flip_dune['blockchain'])]
# Step 4: Union the result with filtered_ch
final_df = pd.concat([combined_flip_dune, filtered_ch])
# final_df


,dt,blockchain,name,layer,num_qualified_txs
0,2024-04-19,blast,Blast,L2,347361.0
1,2024-03-15,blast,Blast,L2,197586.0
2,2024-04-16,blast,Blast,L2,341629.0
3,2024-05-15,blast,Blast,L2,477503.0
4,2024-04-25,blast,Blast,L2,396884.0
...,...,...,...,...,...
18,2024-04-18,metal,Metal,L2,1.0
19,2024-04-03,metal,Metal,L2,1.0
20,2024-04-07,metal,Metal,L2,2.0
21,2024-04-24,metal,Metal,L2,1.0


In [7]:
final_df.to_csv('outputs/unified_qualified_txs.csv', index=False)